In [1]:
import pandas as pd
import re
from tqdm import tqdm
from collections import Counter
import random

# HYBRID

In [2]:
df = pd.read_csv("./appropriateness/Appropriateness.csv")

In [3]:
len(df)

163332

In [5]:
df.columns

Index(['text', 'inappropriate', 'offline_crime', 'online_crime', 'drugs',
       'gambling', 'pornography', 'prostitution', 'slavery', 'suicide',
       'terrorism', 'weapons', 'body_shaming', 'health_shaming', 'politics',
       'racism', 'religion', 'sexual_minorities', 'sexism', 'social_injustice',
       'human_labeled', 'toxic_auto'],
      dtype='object')

In [52]:
threshold = 0
label_name = 'inappropriate'
df_high_quality = df[(df[label_name] >= 1-threshold)|(df[label_name] <=threshold)]
len(df_high_quality)

106072

In [41]:
df_vers1 = pd.read_csv("../Inappropriateness_sensitive_topics_1.0/appropriateness/Appropriateness.csv")

In [4]:
df_vers1.columns

Index(['text', 'inappropriate', 'human_labeled', 'offline_crime',
       'online_crime', 'drugs', 'gambling', 'pornography', 'prostitution',
       'slavery', 'suicide', 'terrorism', 'weapons', 'body_shaming',
       'health_shaming', 'politics', 'racism', 'religion', 'sexual_minorities',
       'sexism', 'social_injustice', 'none', 'toxic_new'],
      dtype='object')

In [5]:
def depersonalize(text):
    
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
    
    return text


In [6]:
df.head()

,text,unsafe,crime_real,crime_web,drugs,gambling,pornography,prostitution,slavery,suicide,...,body_shaming,halth_shaming,politics,racism,religion,sex_minorities,sexism,social,human_labeled,toxic_auto
0,Начальники ДК и нынешнего места работы?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.09
1,"Читаю. Драки, грабежи, ДТП. Изредка убийства, ...",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.18
2,"Допустимо, если информация не нарушает правил ...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.02
3,В больницу при приеме на работу тоже мед комис...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00
4,а зачем ты на проураинские митинги выходил? во...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.40


In [7]:
df['processed'] = df['text'].apply(depersonalize)

In [18]:
isnotequal = []
for i,el in tqdm(df.iterrows(),total = len(df)):
    if el['text'] != el['processed']:
        isnotequal.append(1)
    else:
        isnotequal.append(0)

100%|███████████████████████████████████████████████████████████████████████| 163332/163332 [00:12<00:00, 12769.10it/s]


In [20]:
Counter(isnotequal)

Counter({0: 149329, 1: 14003})

In [21]:
df['motequal'] = isnotequal

In [29]:
for i,el in df[df['motequal']==1].iterrows():
    print(el['text'])
    print(el['processed'])
    print("="*100)
#     if i == 5:
    break

@Andrey Stanin это все конечно имеет логику и прочее, только тут есть один изъян–после смены формации нам же эти завалы нечистот разгребать придется(это уже не говоря о том, что со сменой формации мы можем об*сратся и лозунги солидаризма и прочее станут очень хорошей основой для фашизма),как по мне самая большая проблема сейчас это вот такие "левые" как Попов с его путинизмом и всякие соц демы типо грудинки/плутошкина(про КПРФ с дедушкой Зюгановым я вообще молчу), потому как именно вот такие господа слили революцию в Германии и к власти пришли нацисты, так что как говорится, прежде чем объединятся надо выяснить кто есть кто и исходя из этого размежеватся
 Stanin это все конечно имеет логику и прочее, только тут есть один изъян–после смены формации нам же эти завалы нечистот разгребать придется(это уже не говоря о том, что со сменой формации мы можем об*сратся и лозунги солидаризма и прочее станут очень хорошей основой для фашизма),как по мне самая большая проблема сейчас это вот такие 

In [8]:
df['text'] = df['processed']

In [9]:
del df['processed']
# del df['motequal'] 

# Смотрим на опасность, проверяем очитску от очевидных матов

In [13]:
val = 1
lst = df[df['inappropriate']>=val]['text'].tolist()

# val = 0
# lst = df[df['inappropriate']<=val]['text'].tolist()

random.sample(lst,10)

['Пиши, я не против. Тебе же задали.Если не хочешь писать, иди работать водителем маршрутки или торговкой на рынок. Там такие как ты нужны как раз.',
 'В ее возрасте некоторые в гробу лежат )',
 'Вам никогда не казалось, что Бога нет и миром правит Сатана? Я любил--любовь всегда отвергали. Где был Бог?',
 'Не думалось, что когда-нибудь напишу это..Но, если ты уже закончил школу, подумай о суициде. Человек с такими аналитическими способностями обречен стать бесконечным источником страданий для себя и окружающих(',
 'Каждый вечер прихожу к ограде детского сада, где рядом играют дети, прошу рандомную девочку подойди и достаю член, а затем прошу что бы она его потрогала и своей рукой беру ее руку и надрачиваю себе.',
 'А может и не сожгла, и они устроили тройничок в стиле БДСМ.',
 'NUMBER. Кумар который выворачивает все суставы прокручивая их через мясорубку , невозможность есть, пить , спать . Вытекание жидкостей со всех щелей. Головные боли и высыпания грязи в виде прыщей по всему телу п

# Смотрим мультилейбл

In [14]:
df.columns

Index(['text', 'inappropriate', 'offline_crime', 'online_crime', 'drugs',
       'gambling', 'pornography', 'prostitution', 'slavery', 'suicide',
       'terrorism', 'weapons', 'body_shaming', 'health_shaming', 'politics',
       'racism', 'religion', 'sexual_minorities', 'sexism', 'social_injustice',
       'human_labeled', 'toxic_auto'],
      dtype='object')

In [24]:
topic = 'sexual_minorities'

val = 0.85
lst = df[(df['inappropriate']>=val)&(df[topic]==1)]['text'].tolist()

# val = 0.15
# lst = df[(df['inappropriate']<=val)&(df[topic]==1)]['text'].tolist()

random.sample(lst,10)

['да вы, батенька, знаете толк в извращениях',
 'но банковские работники всегда в курсе, что этоА откуда у банковских работников такая глубокая осведомлённость по вопросу названий гей-сайтов?',
 'Хм... Судя по пользовательской категории это гей бар... Пожалуй воздержусь от обращения к ним...',
 'В одном из оплотов демократии запахло комунизмом... ',
 'Непорядок, почему нет какого нибудь гея ?',
 'ЛГБТ они правильно гоняют. А вот с остальным перебор...',
 'хм... а если я из соломы и тряпки сделаю куклу и черты детские нарисую, меня тоже к педофилу приравняют или к поклоннику вуду?',
 'сам бы так и поступил, жаль конечно за ненужный срок.кстати собирались же педофилам дополнительно к сроку кастрацию делать. опять всё только на словах? :(',
 'они живут NUMBER лет после того как учатся на хирургов, или после первого гомосексуального опыта?',
 'Может все кто такие дела возбуждают страдают от неразделённой любви и мстят "педофилам"']

In [42]:
df_vers1.columns

Index(['text', 'inappropriate', 'human_labeled', 'offline_crime',
       'online_crime', 'drugs', 'gambling', 'pornography', 'prostitution',
       'slavery', 'suicide', 'terrorism', 'weapons', 'body_shaming',
       'health_shaming', 'politics', 'racism', 'religion', 'sexual_minorities',
       'sexism', 'social_injustice', 'none', 'toxic_new'],
      dtype='object')

In [43]:
df.columns

Index(['text', 'unsafe', 'crime_real', 'crime_web', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'halth_shaming', 'politics', 'racism',
       'religion', 'sex_minorities', 'sexism', 'social', 'human_labeled',
       'toxic_auto'],
      dtype='object')

In [44]:
new_columns = ['text', 'inappropriate', 'offline_crime', 'online_crime', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'health_shaming', 'politics', 'racism',
       'religion', 'sexual_minorities', 'sexism', 'social_injustice', 'human_labeled',
       'toxic_auto']

In [45]:
df.columns = new_columns

In [46]:
df.head()

,text,inappropriate,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
0,Начальники ДК и нынешнего места работы?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.09
1,"Читаю. Драки, грабежи, ДТП. Изредка убийства, ...",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.18
2,"Допустимо, если информация не нарушает правил ...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.02
3,В больницу при приеме на работу тоже мед комис...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00
4,а зачем ты на проураинские митинги выходил? во...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.40


In [47]:
df.to_csv("./appropriateness/Appropriateness.csv",index = None)

# MULTILABEL

In [55]:
df_multi_v1 = pd.read_csv("../Inappropriateness_sensitive_topics_1.0/sensitive_topics/sensitive_topics.csv")

In [56]:
df_multi_v1.columns

Index(['text', 'offline_crime', 'online_crime', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'health_shaming', 'politics', 'racism',
       'religion', 'sexual_minorities', 'sexism', 'social_injustice'],
      dtype='object')

In [25]:
df_multi = pd.read_csv("./sensitive_topics/sensitive_topics.csv")

In [26]:
df_multi.columns

Index(['text', 'offline_crime', 'online_crime', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'health_shaming', 'politics', 'racism',
       'religion', 'sexual_minorities', 'sexism', 'social_injustice'],
      dtype='object')

In [57]:
df_multi.columns = df_multi_v1.columns

In [28]:
topic = 'social_injustice'
lst = df_multi[df_multi[topic]==1]['text'].tolist()
random.sample(lst,10)

['Путин войдёт в историю именем, Владимир обнулёный или Владимир ковидный.',
 'О, дак это же как мой город, один в один, нищета и разруха. Только с другой стороны от Москвы.',
 'Пусть жизни хлебнет. Вы просто тряметесь над ним сильно. Я в NUMBER уже пробовал сам жить.',
 'Даже та средняя зарплата в NUMBER тыс руб о которой сейчас говорит власть - это вранье!!! Во всяком случае, где я живу средняя зарплата в основном NUMBER-NUMBER тыс руб......',
 'Помогите решить задачу по нахождению уровня безработицы',
 'Голосуем на выборах,терпилоиды!Умираем в ковидниках!Поодиночке,больше троих не собираться!Всё равно умные уехали остались одни дебилы и мусора.',
 'всегда проигрываю когда нищая карликовая пидораха с борд пишет от имени успешных людей',
 'Нахуй вообще на русскоязычную работать? Ты ебанутый?',
 'Правильнее сказать ублюдки не имеющие мозгов на заработок денег способом отличным от воровства',
 ' Free А до санкций мы так хорошо жили просто зашибись и экономика на высоте была только пол с

In [29]:
len(df_multi)

33303

In [76]:
df_multi.to_csv("./sensitive_topics/sensitive_topics.csv", index = None)

In [80]:
import os
for f in os.listdir("./sensitive_topics/"):
    if ".csv" in f:
        df_curr = pd.read_csv(os.path.join("./sensitive_topics/",f))
        df_curr.columns = df_multi.columns
        df_curr.to_csv(os.path.join("./sensitive_topics/",f),index = None)
#     print(f)